In [8]:
%matplotlib inline


    Football: The football.csv file contains the results from the English Premier League. The columns labeled ‘Goals’ and ‘Goals Allowed’ contain the total number of goals scored for and against each team in that season (so Arsenal scored 79 goals against opponents, and had 36 goals scored against them). Write a program to read the file, then print the name of the team with the smallest difference in ‘for’ and ‘against’ goals.
    Weather: In weather.csv you’ll find daily weather data. Write a program to read the file, then output the day number (column one) with the smallest temperature spread (the maximum temperature is the second column, the minimum the third column).
    See if you can write the same program to solve both questions.


In [ ]:
import pandas as pd
import matplotlib as plt
import csv
import polars as pl
from pathlib import Path
import re
import io
import os
os.chdir('/home/matebarey/Downloads/repos/mozity')

def csv_quick_scan(b: bytes): # cr and lf are control chars  coded 0x0D  (13 decimals) and 0x0A (10 decial) used to mark line break
    crlf = b.count(b"\r\n"); lf = b.count(b"\n") - crlf; cr = b.count(b"\r") - crlf
    first = b.splitlines()[:1]
    # interior BOM (ignore BOM at very start)
    start = 3 if b.startswith(b"\xef\xbb\xbf") else 0
    return {
        "CRLF": crlf, "LF_pure": lf, "CR_pure": cr,
        "cr_only_or_mixed": (cr > 0 and lf == 0 and crlf == 0) or (b.find(b"\r") != -1 and crlf == 0),
        "excel_sep": bool(first and first[0].lower().startswith(b"sep=")),
        "nul": (b"\x00" in b),
        "bom_interior": (b.find(b"\xef\xbb\xbf", start) != -1),
        "guess_delim": max([b",", b";", b"\t", b"|", b":"], key=b.count).decode(),
    }


def load_csv_polars_quick(path, has_header=True):

    b = Path(path).read_bytes()
    s = csv_quick_scan(b)
    print(s)  # one-liner stats/flags BEFORE any fixes

    # tiny transform map
    transforms = {
        "cr_only_or_mixed": lambda x: x.replace(b"\r\n", b"\n").replace(b"\r", b"\n"),
        "bom_interior":     lambda x: x.replace(b"\xef\xbb\xbf", b""),
        "nul":              lambda x: x.replace(b"\x00", b""),
    }
    raw = b
    for k, fn in transforms.items():
        if s[k]: raw = fn(raw)

    opts = {"has_header": has_header, "separator": s["guess_delim"]}
    if s["excel_sep"]: opts["skip_rows"] = 1

    return pl.read_csv(io.BytesIO(raw), **opts)

df = pd.read_csv('football.csv',sep=',')


# dfp = pl.read_csv('football.csv',
#                   separator=',',
#                   has_header=True,
#                   eol_char='\r')  # Mac line ending
dfp = load_csv_polars_quick('football.csv')
dfp

{'CRLF': 0, 'LF_pure': 0, 'CR_pure': 20, 'cr_only_or_mixed': True, 'excel_sep': False, 'nul': False, 'bom_interior': False, 'guess_delim': ','}


Team,Games,Wins,Losses,Draws,Goals,Goals Allowed,Points
str,i64,i64,i64,i64,i64,i64,i64
"""Arsenal""",38,26,9,3,79,36,87
"""Liverpool""",38,24,8,6,67,30,80
"""Manchester United""",38,24,5,9,87,45,77
"""Newcastle""",38,21,8,9,74,52,71
"""Leeds""",38,18,12,8,53,37,66
…,…,…,…,…,…,…,…
"""Bolton""",38,9,13,16,44,62,40
"""Sunderland""",38,10,10,18,29,51,40
"""Ipswich""",38,9,9,20,41,64,36


In [ ]:
df['Net_Goals'] = df['Goals'] - df['Goals Allowed']
#find minimum value of 'Net Goals'
df.min()['Net_Goals']
#use conditional indexing to find
#the row with the minimum value
#and then output the 'Team' column value
df[df['Net_Goals'] == int(df.min()['Net_Goals'])].iloc[0]['Team']
#pandsas


#csv

import csv

def parse_next_line(csv_file):
    for line in csv.DictReader(csv_file):
        yield line

'Leicester'

In [ ]:
# 2) compute Net_Goals without touching Goals
from polars import col as c

dfp = dfp.with_columns(
    Net_Goals = pl.col("Goals") - pl.col("Goals Allowed")
)

min_goals_condn = c.Net_Goals == c.Net_Goals.min()
min_goals = dfp.filter(min_goals_condn).select(c.Net_Goals)
min_goals_team = dfp.filter(min_goals_condn).select(c.Team)

print(f'min_goals: {min_goals}')
print(f'min_goals_team: {min_goals_team}')



min_goals: shape: (1, 1)
┌───────────┐
│ Net_Goals │
│ ---       │
│ i64       │
╞═══════════╡
│ -34       │
└───────────┘
min_goals_team: shape: (1, 1)
┌───────────┐
│ Team      │
│ ---       │
│ str       │
╞═══════════╡
│ Leicester │
└───────────┘


1+99


2. Moving onto weather data


In [82]:
df = pd.read_csv('weather.csv', sep=',')
df_p = load_csv_polars_quick('weather.csv')
df.dtypes

{'CRLF': 0, 'LF_pure': 0, 'CR_pure': 30, 'cr_only_or_mixed': True, 'excel_sep': False, 'nul': False, 'bom_interior': False, 'guess_delim': ','}


Day             int64
MxT             int64
MnT             int64
AvT             int64
AvDP          float64
1HrP TPcpn      int64
PDir            int64
AvSp          float64
Dir             int64
MxS             int64
SkyC          float64
MxR             int64
Mn              int64
R AvSLP       float64
dtype: object

In [ ]:

df['avghl'] = df.apply(lambda row: (row.MxT + row.MnT)/2,axis=1)
df[df['avghl'] == (df.max()['avghl'])].iloc[0]['Day']

np.float64(24.0)

In [85]:
df_p = df_p.with_columns(((c.MxT + c.MnT)/2).alias("avghl"))
df_p

Day,MxT,MnT,AvT,AvDP,1HrP TPcpn,PDir,AvSp,Dir,MxS,SkyC,MxR,Mn,R AvSLP,avghl
i64,i64,i64,i64,f64,i64,i64,f64,i64,i64,f64,i64,i64,f64,f64
1,88,59,74,53.8,0,280,9.6,270,17,1.6,93,23,1004.5,73.5
2,79,63,71,46.5,0,330,8.7,340,23,3.3,70,28,1004.5,71.0
3,77,55,66,39.6,0,350,5.0,350,9,2.8,59,24,1016.8,66.0
4,77,59,68,51.1,0,110,9.1,130,12,8.6,62,40,1021.1,68.0
5,90,66,78,68.3,0,220,8.3,260,12,6.9,84,55,1014.4,78.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
26,97,64,81,70.4,0,50,5.1,200,12,4.0,107,45,1014.9,80.5
27,91,72,82,69.7,0,250,12.1,230,17,7.1,90,47,1009.0,81.5
28,84,68,76,65.6,0,280,7.6,340,16,7.0,100,51,1011.0,76.0


In [ ]:
max_avg_condn = c.avghl == c.avghl.max()
day_with_max_avghl = df_p.filter(max_avg_condn).select(c.Day)
max_avghl = df_p.filter(max_avg_condn).select(c.avghl)
day_with_max_avghl
len(df_p)

import plotly.express as px

def min_net(filename, mxi, mni,loader=None):
    df = loader(filename) if loader else pl.read_csv(filename)   # <-- use loader if given
    df = df.with_columns((mxi - mni).alias("net"))
    first_col = df.columns[0]


    data_dict = df.to_dict(as_series=False )

    #graph net values
    fig = px.bar(data_dict,x=first_col,y='net',title='Net')
    fig.show()
    min_net_condn = c.net == c.net.min()
    result = df.filter(min_net_condn)
    result_count = result.height
    for i in range(result_count):
         print(result.row(i)[df.columns.index(first_col)])


MxT = pl.col("MxT")
MnT = pl.col("MnT")
min_net("weather.csv", MxT, MnT, loader=load_csv_polars_quick)


{'CRLF': 0, 'LF_pure': 0, 'CR_pure': 30, 'cr_only_or_mixed': True, 'excel_sep': False, 'nul': False, 'bom_interior': False, 'guess_delim': ','}


14


In [103]:

goals = pl.col("Goals")
goals_allowed = pl.col("Goals Allowed")
min_net("football.csv",goals, goals_allowed, loader=load_csv_polars_quick)

{'CRLF': 0, 'LF_pure': 0, 'CR_pure': 20, 'cr_only_or_mixed': True, 'excel_sep': False, 'nul': False, 'bom_interior': False, 'guess_delim': ','}


Leicester


In [ ]:
#csv version

def team_min_goal_diff(filename,
                       team_col="Team",
                       for_col="Goals For",       # use "Goals" if your file has "Goals" + "Goals Allowed"
                       against_col="Goals Against"):
    with open(filename, newline="") as f:
        rows = csv.DictReader(f)
        # generator expression: (team, |for - against|)
        pairs = (
            (r[team_col], abs(int(r[for_col]) - int(r[against_col])))
            for r in rows
        )
        team, diff = min(pairs, key=lambda t: t[1])
    return team, diff


def day_max_avg_temp(filename, day_col="Day", hi_col="MxT", lo_col="MnT"):
    def clean_int(s: str) -> int:
        # keep digits and an optional leading minus
        return int(re.sub(r"(?<!^)[^\d]|(?!^-)[^\d-]", "", s))
        # simpler (often enough): int(re.sub(r"[^-0-9]", "", s))

    with open(filename, newline="") as f:
        rows = csv.DictReader(f)
        # generator: (day, (hi+lo)/2)
        pairs = (
            (int(r[day_col]), (clean_int(r[hi_col]) + clean_int(r[lo_col])) / 2)
            for r in rows
        )
        day, avg = max(pairs, key=lambda t: t[1])
    return day, avg

day, avg = day_max_avg_temp("weather.csv", hi_col="MxT", lo_col="MnT")
print(f'day: {day}, avg: {avg}')

team, diff = team_min_goal_diff("football.csv", for_col="Goals", against_col="Goals Allowed")
print(f'team: {team}, diff: {diff}')


day: 24, avg: 83.5
team: Aston_Villa, diff: 1


In [ ]:
import polars as pl
from polars import col as c

def min_goal_diff_polars(filename: str, loader=None):
    """football.csv with Polars (lazy). Returns (team, diff)."""
    lf = pl.scan_csv(filename) if loader is None else pl.from_pandas(loader(filename).to_pandas()).lazy()
    out = (
        lf.with_columns((c("Goals For") - c("Goals Against")).abs().alias("diff"))
          .select(c.Team, c.diff)
          .sort(c.diff)
          .limit(1)
          .collect()  # engine="gpu" optional if you have it
    )
    return out["Team"].item(), out["diff"].item()

def max_avg_temp_polars(filename: str, hi="MxT", lo="MnT", loader=None):
    """weather.csv with Polars (lazy). Returns (day, avg)."""
    lf = pl.scan_csv(filename) if loader is None else pl.from_pandas(loader(filename).to_pandas()).lazy()
    out = (
        lf.with_columns(((c(hi) + c(lo)) / 2).alias("avghl"))
          .select(c.Day, c.avghl)
          .sort(c.avghl, descending=True)
          .limit(1)
          .collect()
    )
    return int(out["Day"].item()), out["avghl"].item()
